## Introduction



The decision to use probabilistic forecasts and stochastic optimization in bidding for power markets, such as the day-ahead electricity market, is a subject of ongoing debate. The most common argument against it is that it adds complexity to an already complex decision making process. There are also more trivial reasons like the absence of scenarios or any kind of uncertainty forecasts. Despite these challenges, the primary advantage of stochastic optimization &ndash; its superior performance when paired with reliable probabilistic forecasts<a href="#footnote_01">*</a> &ndash; is undeniable. There are publications that quantify how much better stochastic optimization is compared to a deterministic approach for unit-commitment and bidding problems. However, what was somewhat unclear to me is when you actually need probabilistic forecasts and which one until I found this paper <a href="#beykirch_2022">[2]</a>. This blog post aims to concisely review the "Bidding Curve Optimization" section from the insightful paper titled "__Bidding and Scheduling in Energy Markets: Which Probabilistic Forecast Do We Need?__".

---
<a name="footnote_02">*</a> It's not that hard, check out my paper for creating time-coupled wind power scenarios <a href="#becker_2018">[1]</a>.

## Approach
The authors dissect the question at hand by analytically examining various use cases, providing a structured approach to the problem. Before being able to do that, we need to first understand the fundamental analytical setup. Furthermore, I will streamline the approach by focusing solely on the power plant owner's perspective, in contrast to the paper's dual focus on both market operators and power plant owners.

Let's have a closer look at the mixed-integer stochastic two stage problem that is used in the paper. In the field of power market bidding, the mixed-integer stochastic two-stage problem stands as a sophisticated modeling approach, offering a nuanced method to capture the complexities of decision-making under uncertainty. This is the generic objective function, in which the parts that concern the market operator view are already removed:
$$
\min_{g^{\omega}, u^{\omega}, s^{\omega}, V(.)} \mathbb{E} \left[
\sum_{t \in T} \pi^{\omega}_{t} V_{t}(\pi^{\omega}_{t}) + c^{F}_{t} g^{\omega}_{t} + c^{ST}_{t} I^{st}_{t} (u^{\omega}_{t}, u^{\omega}_{t-1})
\right]
$$

The first thing that can be noticed is that we are minimizing an expected value over all time steps in $T$.
The first stage decision consists of submitting a bidding curve $V_{t}(\pi^{\omega}_{t})$ with uncertain price realizations $\pi^{\omega}_{t}$ to the market. Next comes the fuel costs $c^{F}_{t}$ which are multiplied with the plant's power generation $g^{\omega}_{t}$. This is followed by the start-up costs $c^{ST}_{t}$ as well as the function $I^{st}_{t}$, which becomes 1 when the power plant starts up in $t$.

The constraints look like this:
\begin{aligned}
& \text{s.t.} \\
& g^{\omega}_{t} + V_{t}(\pi^{\omega}_{t}) = 0, \\
& u^{\omega}_{t} \underline{g}_{t} \leq g^{\omega}_{t} \leq u^{\omega}_{t} \overline{g}_{t}, \\
& V_{t}(\cdot) \in V, \; u^{\omega}_{t} \in \{0, 1\} \quad \forall t \in T, \quad \text{a.e.} \; \omega \in \Omega.
\end{aligned}
I have omitted the storage component because I won't look at it in more detail. The first constraint is some kind of power balance. Whatever the bidding curve is returning for a given price needs to be produced. The second constraint makes sure that the power plant is operating within its operational limits.

## Use cases

Analyzing different use cases of power plants reveals how their characteristics influence bidding strategies and the need for probabilistic forecasting. Let's examine these scenarios, starting with the simplest case.

### Power plant without start-up costs
In this first basic case we assume that we have a power plant without start-up costs and that we simply want to bid everything we have into the market. In this particular case we can substitute our generation $g^{\omega}_{t}$ with the bidding curve $V_{t}(\pi^{\omega}_{t})$ because we will produce whatever the market clearing is telling us. This boils down to this objective function:
$$
\min_{V(.)} \sum_{t \in T} \mathbb{E} \left[
 (\pi^{\omega}_{t} -  c^{F}_{t}) V_{t}(\pi^{\omega}_{t})
\right]
$$
, which could be rewritten when minimizing over all scenarios $S$ with their weights $w_s$ and prices $\pi_{t,s}$:
$$
\min_{V(.)} \sum_{s \in S} w_s \sum_{t \in T} 
 (\pi_{t,s} -  c^{F}_{t}) V_{t}(\pi_{t,s})
$$

Due to the characteristics of a bidding curve $V(.)$, which are that we don't bid anything below our productions costs and all our production above, it can be concluded that we do not need a forecast for bidding in this case.

### Power plant with start-up costs

When adding start-up costs our optimization problem becomes:
$$
\min_{V(.), \mathbf{g}, \mathbf{u}} \sum_{s \in S} w_s \sum_{t \in T}  
 (\pi_{t,s} -  c^{F}_{t}) V_{t}(\pi_{t,s}) + c^{ST} I^{ST}(u_{t,s}, u_{t-1})
$$

\begin{aligned}
& \text{s.t.} \\
& g_{ts} + V_{t}(\pi_{t,s}) = 0, \\
& u_{t,s} \underline{g}_{t} \leq g^{\omega}_{t} \leq u_{t,s} \overline{g}_{t}, \\
& V_{t}(\cdot) \in V, \; u^{\omega}_{t} \in \{0, 1\} \quad \forall t \in T, \forall s \in S.
\end{aligned}

In contrast to the problem without start-up costs, we now have to deal with the integer variables $u_{t,s}$ and the start-up function $I^{ST}$ that links two adjacent time steps. This is also why pairwise joint predictive distributions are needed. This seems intuitive when looking at the equations at hand but I still find it interesting because basically everyone is going directly from deterministic forecasts to scenarios that were derived from the full joint distribution. However, you could also take a small step and simply estimate pairwise joint predictive functions which should be easier than for example dealing with copulas to get your scenarios right.

### Power plant with storage

I will not get into detail here because it is rather obvious that you are going to need a full joint distribution as soon as your model has a storage. The reason is that not only adjacent time steps depend on each other but basically all. If you want to know more about this, the paper offers a detailed explanation.

## Bidding with variable time step coupling

At the end of the paper, the authors run an actual simulation in which they model a power plant with start-up costs and compare deterministic forecasts and probabilistic forecasts in combination with stochastic optimization. There are two findings that I find worth highlighting here.

### Share of start-up costs

The difference between having and not having start-up costs is $c^{ST} I^{ST}(u_{t,s}, u_{t-1})$ in the objective function. Hence, as larger the share of your start-up costs of your total costs is, the more sense it makes to use scenarios. This is also one main takeaway for me. Whether stochastic optimization really yields noteworthy better results depends a lot on the input parameters, i.e. your power plant and costs.

In the paper, we can see diverging results when the share of start-up costs exceeds 10 %. In practical terms, this suggests that power plants with significant start-up costs might find substantial benefits in adopting stochastic optimization to manage these expenses more effectively.

It would have been interesting to see a similar comparison for the storage case. If the storage is rather small compared to the rest of the system, the advantage of stochastic optimization should also be smaller.

### Correlation

This point is also rather obvious but worth mentioning. What the authors show in their case study is how the correlation between the prices from time step to time step is impacting the superiority of stochastic optimization. When prices are completely uncorrelated, stochastic optimization won't help at all. The result will be as good/bad as using the mean value as a forecast. However, the higher the correlation gets, i.e. the stronger the patterns are between prices, the better the stochastic approach gets.

The main takeaway for me here is once again that input data matters.


## Summary
If you have asked yourself when to use stochastic optimization for bidding into energy markets and what kind of probabilistic forecasts are required, you can now find the answer in this great paper <a href="#beykirch_2022">[2]</a> from Mario Beykirch, Tim Janke and Florian Steinke.
They are going through different use cases and demonstrate analytically which forecast are needed. My main takeaways are:

- Even in the face of uncertain price outcomes, the necessity of forecasts can vary significantly, demonstrated by the case of a simple power plant without start-up costs where no forecast might be required.
- Introducing start-up costs changes the dynamic, necessitating only pairwise joint price distributions instead of the full joint distribution, challenging the common assumption that the full joint distribution is always required.
- Correlation matters. The higher the correlation between price hours is, the better a stochastic optimization will be.
- Have a look at the share of start-up costs or more general every term that requires to use probabilistic instead of deterministic forecasts. The lower the contribution to the objective function is (for example a low share of start-up costs), the lower the benefit of a stochastic approach will be.

## Bibliography 
<a name="becker_2018">[1]</a>: R. Becker, "Generation of Time-Coupled Wind Power Infeed Scenarios Using Pair-Copula Construction", in IEEE Transactions on Sustainable Energy, vol. 9, no. 3, pp. 1298-1306, July 2018, doi: [10.1109/TSTE.2017.2782089](https://ieeexplore.ieee.org/document/8171737).

<a name="beykirch_2022">[2]</a>: M. Beykirch, T. Janke & F. Steinke, "Bidding and Scheduling in Energy Markets: Which Probabilistic Forecast Do We Need?", in 17th International Conference on Probabilistic Methods Applied to Power Systems (PMAPS 2022), virtual Conference, 2022, doi: [https://doi.org/10.48550/arXiv.2203.13159](https://doi.org/10.48550/arXiv.2203.13159).
